In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')


In [3]:
#!pip install tokenizers

In [1]:
from gensim.models import Word2Vec, KeyedVectors

In [4]:
#!pip install transformers

In [5]:
from pathlib import Path
paths = [str(x) for x in Path('./dataset/').glob('mydataset.csv')]

In [6]:
#from pathlib import Path
#paths = [str(x) for x in Path('/content/drive/MyDrive/BERT_build_newmodel/data/').glob('**/*.txt')]

In [7]:
paths

['dataset/mydataset.csv']

In [8]:
# Initialize a tokenizer
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()

In [9]:
# Customize training
tokenizer.train(files=paths[:5], vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>'," ",'',' ', '<unk>', '<mask>'])
#OR
#tokenizer.train(files=path,
                #vocab_size=50265,
                #min_frequency=2,
                #special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])

In [10]:
# Save files to disk
!mkdir MyRoBERTaa
tokenizer.save_model("MyRoBERTaa")
#OR
#!mkdir -p "models/MyRoBERTaa"
#tokenizer.save("models/MyRoBERTaa"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['MyRoBERTaa/vocab.json', 'MyRoBERTaa/merges.txt']

In [11]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./MyRoBERTaa/vocab.json",
    "./MyRoBERTaa/merges.txt",
)

In [12]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [13]:
tokenizer.encode("አበበ በሶ በላ ።")

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [14]:
tokenizer.encode("አበበ በሶ በላ ።").tokens

['<s>', 'áĬłáīłáīł', 'ĠáīłáĪ¶', 'ĠáīłáĪĭ', 'Ġáį', '¢', '</s>']

In [15]:
tokenizer.decoder

In [16]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [17]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("./MyRoBERTaa", max_len=512)

In [18]:
from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

In [19]:
model.num_parameters()

83504416

In [20]:
%%time
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./dataset/mydataset.csv",
    block_size = 64
)

/home/neima/.local/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 11 s, sys: 1.82 s, total: 12.9 s
Wall time: 3.28 s


In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [22]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./MyRoBERTaa",
    overwrite_output_dir=True,
    num_train_epochs=200,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [23]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/home/neima/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 33297
  Num Epochs = 200
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 104200
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,7.407300
1000,6.531500
1500,6.277400
2000,6.116200
2500,5.993800
3000,5.838300
3500,5.740600
4000,5.587900
4500,5.460300
5000,5.314700


Saving model checkpoint to ./MyRoBERTaa/checkpoint-10000
Configuration saved in ./MyRoBERTaa/checkpoint-10000/config.json
Model weights saved in ./MyRoBERTaa/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./MyRoBERTaa/checkpoint-20000
Configuration saved in ./MyRoBERTaa/checkpoint-20000/config.json
Model weights saved in ./MyRoBERTaa/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to ./MyRoBERTaa/checkpoint-30000
Configuration saved in ./MyRoBERTaa/checkpoint-30000/config.json
Model weights saved in ./MyRoBERTaa/checkpoint-30000/pytorch_model.bin
Deleting older checkpoint [MyRoBERTaa/checkpoint-10000] due to args.save_total_limit
Saving model checkpoint to ./MyRoBERTaa/checkpoint-40000
Configuration saved in ./MyRoBERTaa/checkpoint-40000/config.json
Model weights saved in ./MyRoBERTaa/checkpoint-40000/pytorch_model.bin
Deleting older checkpoint [MyRoBERTaa/checkpoint-20000] due to args.save_total_limit
Saving model checkpoint to ./MyRoBERTaa/checkpoint-50000
C

CPU times: user 5h 9min 30s, sys: 1min 2s, total: 5h 10min 33s
Wall time: 5h 9min 20s


TrainOutput(global_step=104200, training_loss=0.9368861754430232, metrics={'train_runtime': 18560.9699, 'train_samples_per_second': 358.785, 'train_steps_per_second': 5.614, 'total_flos': 8.075957163477715e+16, 'train_loss': 0.9368861754430232, 'epoch': 200.0})

In [30]:
trainer.save_model("./MyRoBERTaa")

Saving model checkpoint to ./MyRoBERTaa
Configuration saved in ./MyRoBERTaa/config.json
Model weights saved in ./MyRoBERTaa/pytorch_model.bin


In [35]:
# import pandas as pd
# df = pd.read_csv('MyRoBERTaa_log.tsv',delimiter='\t')
# df.head()

In [36]:
# import matplotlib.pyplot as plt
# plt.title('Training Los')
# plt.xlabel('Steps')
# plt.ylabel('Loss')
# plt.plot(df['Step '], df['Training Loss'])
# plt.show()

In [37]:
# Model paths
#EMODEL_TYPE = "roberta" #@param ["roberta", "bert"]
#MODEL_DIR = "MyRoBERTaa" #@param {type: "string"}
#OUTPUT_DIR = "MyRoBERTaa/output" #@param {type: "string"}
#TRAIN_PATH = "data/dataset.txt" #@param {type: "string"}
#EVAL_PATH = "data/deataset.txt" #@param {type: "string"

In [38]:
# Arguments for training from scratch. I turn off evaluate_during_training,
#   line_by_line, should_continue, and model_name_or_path.
#train_params = {
    #"output_dir": OUTPUT_DIR,
    #"model_type": MODEL_TYPE,
    #"config_name": MODEL_DIR,
    #"tokenizer_name": MODEL_DIR,
    #"train_path": TRAIN_PATH,
    #"eval_path": EVAL_PATH,
    #"do_eval": "--do_eval",
   # "evaluate_during_training": "",
   # "line_by_line": "",
    #"should_continue": "",
    #"model_name_or_path": "",
#}

In [39]:
#from transformers import pipeline
#unmask = pipeline('fill-mask')


In [40]:
#unmask.tokenizer.mask_token
#fill_mask.tokenizer.model

In [2]:
import re
import hm
import pandas as pd


@@@@ This is HornMorpho, version 4.3 @@@@



In [3]:
#fill_mask('የሚዲያ ዲፓርትመንቱም የተዋጣለትና በብቁ የሰው <mask> የተደራጀ ነው')

In [4]:
sentt='እየሄደ ሳለ እናቱን ተመለከታት'
sent=sentt.split() 
result_sent=""
#print(sent)
#result_sent=""
for word in sent: 
    root=hm.anal('amh', word, um=True) 
    if root!=[]: 
        roots=root[0]['lemma'] 
        roots=roots.replace("|", "/") 
        if "/" in roots: 
            result = re.search('(.*)/', roots) 
            #if result not in amharicstop and result not in spch: 
            result_sent+=result.group(1)+" " 
        else: 
            result_sent+=word+" "                         
    else: 
        result_sent+=word+" " 
sent=result_sent
print(sent)

Loading Amharic data from amh_lang
Loading FSTs for አማርኛ (analysis/generation) ...
ሄደ ሳለ እናት ተመለከተ 


In [ ]:
result =  senttt.replace(det, "<mask>")
result

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./MyRoBERTaa",
    tokenizer="./MyRoBERTaa"
)

In [182]:
#sentt='በአስተያየቱ <mask> ለውጥ ተጋ ምን በላይ መሰረታዊ ነጥብ ነው'
predictions = fill_mask(sent)
#print(prediction)
x=0
print("Top 5 senses :")
for i in range(5):
    if x>0:
        print(predictions[i]['token_str'])
    x+=1

Top 5 senses :
 እያለ
 ገና
 አልአዛር
 ጠበቀ
